<a href="https://colab.research.google.com/github/Yyyyyjkim/review_analysis/blob/master/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!apt-get update
!apt install chromium-chromedriver
!pip install requests
!pip install bs4
!pip install selenium

In [0]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [0]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
# driver = webdriver.Chrome('chromedriver')

In [27]:
driver.get("https://smartstore.naver.com/jainystore/products/2455969597")
driver.find_element_by_xpath("//div[@class='detail_tab_floatable']//li[@class='item']/a[@class='link N=a:tab.review']").click()
time.sleep(5)

KeyboardInterrupt: ignored

In [0]:
div_class = 'header_review _review_list_header'
ui_class = 'sort_list _review_list_header_sort'
ranking = '랭킹순'
date = '최신순'
desc = '평점 높은순'
asc = '평점 낮은순'
sorting = [ranking,date,desc,asc]
sorting_names = ['ranking','date','desc','asc']
path = [f"//div[@class='{div_class}']//ul[@class='{ui_class}']//a[text()='{i}']" for i in sorting]

In [0]:
for i in range(len(path)):
    print(i)
    driver.find_element_by_xpath(path[i]).click()
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_page = soup.find_all('p',class_='review_text _review_text')
    globals()['review_{}'.format(sorting_names[i])] = [each_line.get_text().strip() for each_line in review_page[:20]]
    for page in range(2,11):
        path_page = f"//nav[@class='module_pagination _review_list_page']//a[text()='{page}']"
        driver.find_element_by_xpath(path_page).click()
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_page = soup.find_all('p',class_='review_text _review_text')
        review = [each_line.get_text().strip() for each_line in review_page[:20]]
        globals()['review_{}'.format(sorting_names[i])].extend(review)

0
1
2
3


In [0]:
import pickle
import pandas as pd

In [0]:
ranking = pd.DataFrame(review_ranking)
date = pd.DataFrame(review_date)
desc = pd.DataFrame(review_desc)
asc = pd.DataFrame(review_asc)

In [0]:
review = pd.concat([ranking,date,desc,asc],axis=1)

In [0]:
with open('/content/drive/My Drive/comment_analysis/review.pickle','wb') as f:
  pickle.dump(review,f)